# Handout 17

# **Semáforos**

## **1. Introdução aos Semáforos**

Semáforos são mecanismos de sincronização amplamente utilizados em sistemas operacionais para controlar o acesso de múltiplas threads ou processos a recursos compartilhados. Eles ajudam a evitar condições de corrida (*race conditions*) e garantem que as operações ocorram em uma ordem segura e previsível.

---

## **2. Conceito de Rendez-vous**

O termo *rendez-vous* refere-se a um ponto de encontro onde duas ou mais threads ou processos sincronizam suas execuções. Nesse contexto, as threads devem esperar umas pelas outras em determinados pontos do código antes de prosseguir, garantindo que certas operações sejam realizadas em uma ordem específica.

**Exemplo de Rendez-vous:**

Imagine duas threads, A e B, que devem executar as seguintes etapas:

- **Thread A:**
  1. Executa a tarefa A1.
  2. Espera pela conclusão da tarefa B1.
  3. Executa a tarefa A2.

- **Thread B:**
  1. Executa a tarefa B1.
  2. Espera pela conclusão da tarefa A1.
  3. Executa a tarefa B2.

Para implementar esse comportamento, utilizamos semáforos para sincronizar as threads nos pontos desejados.

---

## **3. Semáforos POSIX**

A biblioteca POSIX fornece uma API padrão para manipulação de semáforos em sistemas Unix-like. Os semáforos podem ser classificados em:

- **Semáforos Binários:** Funcionam como uma trava simples, permitindo que apenas uma thread acesse um recurso por vez.
- **Semáforos Contadores:** Permitem que um número definido de threads acessem o recurso simultaneamente.

### **3.1. Funções Principais**

- **`sem_init`**: Inicializa um semáforo.
- **`sem_wait`**: Decrementa (bloqueia) o semáforo. Se o valor do semáforo for zero, a thread é bloqueada até que o semáforo seja incrementado.
- **`sem_post`**: Incrementa (libera) o semáforo. Se houver threads bloqueadas, uma delas é desbloqueada.
- **`sem_destroy`**: Destroi o semáforo e libera os recursos associados.

---

## **4. Exemplo Prático: Implementando um Rendez-vous com Semáforos**

**Objetivo:** Sincronizar duas threads de modo que cada uma espere a outra completar sua primeira tarefa antes de prosseguir para a segunda.

**Código:**


In [ ]:

#include <pthread.h>
#include <semaphore.h>
#include <stdio.h>

sem_t sem_A;
sem_t sem_B;

void* thread_A(void* arg) {
    printf("Thread A: Executando A1\n");
    sem_post(&sem_A); // Sinaliza que A1 foi concluído
    sem_wait(&sem_B); // Espera B1 ser concluído
    printf("Thread A: Executando A2\n");
    return NULL;
}

void* thread_B(void* arg) {
    printf("Thread B: Executando B1\n");
    sem_post(&sem_B); // Sinaliza que B1 foi concluído
    sem_wait(&sem_A); // Espera A1 ser concluído
    printf("Thread B: Executando B2\n");
    return NULL;
}

int main() {
    pthread_t tA, tB;

    sem_init(&sem_A, 0, 0);
    sem_init(&sem_B, 0, 0);

    pthread_create(&tA, NULL, thread_A, NULL);
    pthread_create(&tB, NULL, thread_B, NULL);

    pthread_join(tA, NULL);
    pthread_join(tB, NULL);

    sem_destroy(&sem_A);
    sem_destroy(&sem_B);

    return 0;
}

**Explicação:**

- Inicialização: Dois semáforos (sem_A e sem_B) são inicializados com valor zero.
- Thread A:
    - Executa a tarefa A1.
    - Chama sem_post(&sem_A) para sinalizar que A1 foi concluída.
    - Chama sem_wait(&sem_B) para esperar que B1 seja concluída.
    - Executa a tarefa A2.
- Thread B:
    - Executa a tarefa B1.
    - Chama sem_post(&sem_B) para sinalizar que B1 foi concluída.
    - Chama sem_wait(&sem_A) para esperar que A1 seja concluída.
    - Executa a tarefa B2.
- Finalização: Após a conclusão das threads, os semáforos são destruídos para liberar os recursos.

## **5. Barreiras**

Barreiras são mecanismos de sincronização que fazem com que um conjunto de threads espere até que todas alcancem um determinado ponto antes de prosseguir. Diferentemente dos semáforos, que podem ser usados para sincronizar pares de threads, as barreiras são úteis para sincronizar grupos maiores.

**Funções Principais**

- pthread_barrier_init: Inicializa uma barreira.
- pthread_barrier_wait: Faz com que a thread espere até que todas as threads atinjam a barreira.
- pthread_barrier_destroy: Destroi a barreira e libera os recursos associados.

Exemplos de uso:

In [ ]:
#include <pthread.h>
#include <stdio.h>

#define NUM_THREADS 5

pthread_barrier_t barreira;

void* tarefa(void* arg) {
    int id = *(int*)arg;
    printf("Thread %d: Antes da barreira\n", id);
    pthread_barrier_wait(&barreira);
    printf("Thread %d: Depois da barreira\n", id);
    return NULL;
}

int main() {
    pthread_t threads[NUM_THREADS];
    int ids[NUM_THREADS];

    pthread_barrier_init(&barreira, NULL, NUM_THREADS);

    for (int i = 0; i < NUM_THREADS; i++) {
        ids[i] = i;
        pthread_create(&threads[i], NULL, tarefa, &ids[i]);
    }

    for (int i = 0; i < NUM_THREADS; i++) {
        pthread_join(threads[i], NULL);
    }

    pthread_barrier_destroy(&barreira);

    return 0;
}


**Explicação:**

- Inicialização: A barreira é inicializada para sincronizar NUM_THREADS threads.
- Thread: Cada thread imprime uma mensagem antes e depois da barreira. A chamada pthread_barrier_wait faz com que a thread espere até que todas as threads alcancem esse ponto.
- Finalização: Após a conclusão das threads, a barreira é destruída para liberar os recursos.